<a href="https://colab.research.google.com/github/VIVEKGANGA/ML_Learning/blob/master/Transfer_Learning_and_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
try:
  %tensorflow_version 2.x
except:
  pass

import tensorflow as tf
import os
import zipfile
from tqdm import tqdm_notebook
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

%matplotlib inline
tf.__version__

'2.2.0'

In [6]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O ./cats_and_dogs_filtered.zip

--2020-05-25 06:35:50--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 2a00:1450:400c:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   220MB/s    in 0.3s    

2020-05-25 06:35:51 (220 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



Unzipping data

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"
zip_object = zipfile.ZipFile(dataset_path, mode="r")

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

Setting up dataset path

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"
train_dir = os.path.join(dataset_path_new,"train")
validation_dir = os.path.join(dataset_path_new, "validation")

Creating data generator

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [13]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [14]:
valid_generator = data_gen_train.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


Building the model

In [0]:
IMG_SHAPE = (128,128,3)

In [17]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [18]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [0]:
base_model.trainable = False

Defining the custom head for our model

In [21]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [23]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(global_average_layer)

Defining model

In [0]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

Compiling Model

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

Training the model

In [31]:
model.fit_generator(train_generator, epochs=10, validation_data=valid_generator)

Epoch 1/10
16/16 [==============================] - 9s 577ms/step - loss: 0.3877 - accuracy: 0.8610 - val_loss: 0.3667 - val_accuracy: 0.8680
Epoch 2/10
16/16 [==============================] - 8s 513ms/step - loss: 0.3370 - accuracy: 0.8855 - val_loss: 0.3267 - val_accuracy: 0.8850
Epoch 3/10
16/16 [==============================] - 8s 511ms/step - loss: 0.3013 - accuracy: 0.9000 - val_loss: 0.2945 - val_accuracy: 0.9040
Epoch 4/10
16/16 [==============================] - 8s 511ms/step - loss: 0.2721 - accuracy: 0.9115 - val_loss: 0.2675 - val_accuracy: 0.9150
Epoch 5/10
16/16 [==============================] - 8s 518ms/step - loss: 0.2481 - accuracy: 0.9215 - val_loss: 0.2451 - val_accuracy: 0.9230
Epoch 6/10
16/16 [==============================] - 8s 512ms/step - loss: 0.2281 - accuracy: 0.9290 - val_loss: 0.2266 - val_accuracy: 0.9250
Epoch 7/10
16/16 [==============================] - 8s 521ms/step - loss: 0.2113 - accuracy: 0.9345 - val_loss: 0.2111 - val_accuracy: 0.9300
Epoch 

Transfer Learning model evaluation

In [32]:
vaid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [33]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9399999976158142


Fine Tuning

In [0]:
base_model.trainable = True

In [35]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 155


In [0]:
fine_tune_at = 120

In [0]:
for layers in base_model.layers[:fine_tune_at]:
  base_model.trainable = False

Compiling the model for fine tuning

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [37]:
model.fit_generator(train_generator, epochs=10, validation_data=valid_generator)

Epoch 1/10
16/16 [==============================] - 10s 626ms/step - loss: 0.1538 - accuracy: 0.9375 - val_loss: 0.0756 - val_accuracy: 0.9700
Epoch 2/10
16/16 [==============================] - 9s 578ms/step - loss: 0.0148 - accuracy: 0.9995 - val_loss: 0.0785 - val_accuracy: 0.9690
Epoch 3/10
16/16 [==============================] - 9s 580ms/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.0868 - val_accuracy: 0.9700
Epoch 4/10
16/16 [==============================] - 9s 578ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.1105 - val_accuracy: 0.9710
Epoch 5/10
16/16 [==============================] - 9s 588ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.1380 - val_accuracy: 0.9690
Epoch 6/10
16/16 [==============================] - 9s 593ms/step - loss: 5.8270e-04 - accuracy: 1.0000 - val_loss: 0.1164 - val_accuracy: 0.9700
Epoch 7/10
16/16 [==============================] - 9s 582ms/step - loss: 2.3450e-04 - accuracy: 1.0000 - val_loss: 0.0838 - val_accuracy: 0.97

Evaluating the fine tune model

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [39]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9679999947547913
